<a href="https://colab.research.google.com/github/NeskaCleo/MonicaGlez/blob/main/Extracci%C3%B3n%2C%20transformaci%C3%B3n%20y%20carga%20de%20datos%20(ETL)/Programaci%C3%B3n_Subway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade google-cloud-storage
import os
from google.cloud import storage
import json
import pandas as pd
import numpy as np

from google.cloud.storage import Blob

from datetime import date
import datetime
import time

import requests

!pip install aspose-cells
import jpype
import asposecells

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'credencialcav.json'

In [ ]:
def getFilesList(_carpeta):

    client = storage.Client()
    bucket = client.bucket('practica_etl')

    blobs = client.list_blobs('practica_etl', prefix=_carpeta)

    lista = []

    for blob in blobs:
        lista.append(blob.name)

    return lista

def getFile(_route):

    client = storage.Client()

    bucket = client.bucket('practica_etl')

    blob = bucket.blob(_route)

    content = blob.download_as_string()

    return content

In [ ]:
url = 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM'
json_1 = requests.get(url).text


In [ ]:
data = json.loads(json_1)

In [ ]:
df_datosanyo_a = pd.json_normalize(data, record_path =['Data'], meta=['COD'])
df_datosvari_a = pd.json_normalize(data, record_path =['MetaData'], meta=['COD'])


In [ ]:
df_datosanyo = df_datosanyo_a.pivot(index = 'COD', columns= 'Anyo', values = 'Valor' )
df_datosvari = df_datosvari_a.pivot(index = 'COD', columns= 'Variable.Nombre', values = 'Nombre' )

In [ ]:
dftot = pd.merge(df_datosvari,df_datosanyo, left_on='COD', right_on='COD')

In [ ]:
dftot

In [ ]:
dflimp = dftot

In [ ]:
dflimp.drop(['Continentes', 'Países y Continentes','Tipo de dato', 'Zonas Geográficas del Resto del Mundo'], axis = 1, inplace=True)

In [ ]:
print(dflimp[dflimp['Sexo']=='Total'])

In [ ]:
dflimp

In [ ]:
dflimp1=dflimp[dflimp['Sexo']=='Total']

In [ ]:
dflimp1

In [ ]:
dflimp1.drop(['Sexo', 'Provincias'], axis = 1, inplace=True)

In [ ]:
dflimp1

In [ ]:
dflimp2 = dflimp1[dflimp1.Municipios.notnull()]# Eliminamos las poblaciones vacias

In [ ]:
dflimp2

In [ ]:
dflimp3 = dflimp2[dflimp2.Países.notnull()]# Eliminamos los paises vacios

In [ ]:
dflimp3

In [ ]:
print(dflimp3[dflimp3['Municipios']=='Vilopriu'])
# Con esto comprobamos que los datos concuerdan. En este caso el valor extranjero sería el total de el resto valores de paises menos España.

In [ ]:
dflimp4 = dflimp3[dflimp3['Países']!='Extranjero']# Hemos hecho un filtrado y ahora tengo los datos limpios sin el valor extranjero

In [ ]:
print(dflimp4[dflimp4['Municipios']=='Vilopriu'])

In [ ]:
dflimp4

In [ ]:
dflimp5= dflimp4.fillna(0)

In [ ]:
dflimp5

In [ ]:
jslimp5= dflimp5.to_json (orient = 'index' , force_ascii= False)

In [ ]:
def saveGcloud(_data, _name, _ruta):

    client = storage.Client()
    bucket = client.bucket('practica_etl')

    blob = bucket.blob(_ruta + _name)
    blob.upload_from_string(_data)

    print('Subido correctamente')

In [ ]:
ruta = 'datoslimpios/poblacion/'
archivo = 'Girona.json'

saveGcloud(jslimp5, archivo, ruta)

In [ ]:
jslimp5

In [ ]:
# Archivo CSV

In [ ]:
df_raw = pd.read_csv('/content/drive/MyDrive/Proyecto_final_ETL/TFA - Proyecto 02 - Postal codes2.csv',sep = ";", encoding='utf-8')


In [ ]:
df_raw

In [ ]:
prov=df_raw["provincia"].str.split('/', expand=True)

In [ ]:
prov

In [ ]:
prov.columns = ['prov_loc', 'Provincia']

In [ ]:
prov.columns

In [ ]:
df = pd.concat([df_raw, prov], axis=1)
df

In [ ]:
pobl=df["poblacion"].str.split('/', expand=True)

In [ ]:
pobl

In [ ]:
pobl.columns = ['ciudad_loc', 'ciudad_trf']

In [ ]:
df1 = pd.concat([df, pobl], axis=1)
df1

In [ ]:
import numpy as np

In [ ]:
df1['provNueva']= np.where(df1['Provincia'].isnull(), df1['prov_loc'], df1['Provincia'] )

In [ ]:
df1['poblNueva']= np.where(df1['ciudad_trf'].isnull(), df1['ciudad_loc'], df1['ciudad_trf'] )

In [ ]:
df1

In [ ]:
df1.drop(['provincia', 'poblacion', 'Provincia', 'ciudad_trf'], axis = 1, inplace=True)

In [ ]:
df1

In [ ]:
df1.columns

In [ ]:

df1.columns = ['Código Postal', 'Provincia_loc', 'Ciudad_loc', 'Provincia', 'Poblacion']
df1.columns

In [ ]:
df1

In [ ]:
df1limp= df1.to_json (orient = 'index' , force_ascii= False)

In [ ]:
ruta = 'datoslimpios/provincia/'
archivo = 'Provincia.json'

saveGcloud(df1limp, archivo, ruta)

In [ ]:
# datos gooble

In [ ]:
import csv
#
with open('/content/drive/MyDrive/Proyecto_final_ETL/URL_JSON.csv', newline='') as File:
    reader = csv.reader(File)
    next(reader, None)
    for row in reader:
        print(row[0])

In [ ]:
cont = 0
import csv
#Datos de origen, fichero CSV con filas de URLS del INEN.
with open('/content/drive/MyDrive/Proyecto_final_ETL/URL_JSON.csv', newline='') as File:
    reader = csv.reader(File)
    next(reader, None)
    for row in reader:
            print(cont)
            archivo = 'población_' + str(cont) + '.json'
            url1= row[0]


            json_1 = requests.get(url1).text

            data = json.loads(json_1)

            df_datosanyo_a = pd.json_normalize(data, record_path =['Data'], meta=['COD'])
            df_datosvari_a = pd.json_normalize(data, record_path =['MetaData'], meta=['COD'])

            df_datosanyo = df_datosanyo_a.pivot(index = 'COD', columns= 'Anyo', values = 'Valor' )
            df_datosvari = df_datosvari_a.pivot(index = 'COD', columns= 'Variable.Nombre', values = 'Nombre' )
            df_datosvari_2 = df_datosvari.reindex(columns=['Municipios','Sexo','Nacionalidad','Totales de edad'])
            dftot = pd.merge(df_datosvari_2,df_datosanyo, left_on='COD', right_on='COD')

            dflimp = dftot

            dflimp1=dflimp[dflimp['Sexo']=='Total']
            dflimp1=dflimp1[dflimp1['Totales de edad']=='Todas las edades']

            dflimp1.drop(['Sexo','Totales de edad'], axis = 1, inplace=True)

            dflimp2 = dflimp1[dflimp1.Municipios.notnull()]# Eliminamos las poblaciones vacias
            dflimp3=dflimp2[dflimp2['Nacionalidad']!='Total']
            dflimp4= dflimp3.fillna(0)

            jslimp4= dflimp4.to_json (orient = 'index' , force_ascii= False)

            ruta = 'datoslimpios/poblacion/'


            saveGcloud(jslimp4, archivo, ruta)
            cont+=1

In [ ]:
cont = 0

archivo = 'población_' + str(cont) + '.json'
url1='https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33723?tip=AM'


json_1 = requests.get(url1).text

data = json.loads(json_1)

df_datosanyo_a = pd.json_normalize(data, record_path =['Data'], meta=['COD'])
df_datosvari_a = pd.json_normalize(data, record_path =['MetaData'], meta=['COD'])

In [ ]:
data

In [ ]:
df_datosanyo = df_datosanyo_a.pivot(index = 'COD', columns= 'Anyo', values = 'Valor' )
df_datosvari = df_datosvari_a.pivot(index = 'COD', columns= 'Variable.Nombre', values = 'Nombre' )
df_datosvari_2 = df_datosvari.reindex(columns=['Municipios','Sexo','Nacionalidad','Totales de edad'])
dftot = pd.merge(df_datosvari_2,df_datosanyo, left_on='COD', right_on='COD')

dflimp = dftot

dflimp1=dflimp[dflimp['Sexo']=='Total']
dflimp1=dflimp1[dflimp1['Totales de edad']=='Todas las edades']

dflimp1.drop(['Sexo','Totales de edad'], axis = 1, inplace=True)

dflimp2 = dflimp1[dflimp1.Municipios.notnull()]# Eliminamos las poblaciones vacias
dflimp3=dflimp2[dflimp2['Nacionalidad']!='Total']
dflimp4= dflimp3.fillna(0)

#jslimp4= dflimp4.to_json (orient = 'index' , force_ascii= False)

#ruta = 'datoslimpios/poblacion/'



In [ ]:
dflimp4